In [1]:
import pandas as pd

import numpy as np
import faiss
import pickle
import os
from sentence_transformers import SentenceTransformer




c:\Users\91860\Desktop\Assessment_recommendation_System\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv(r"products_cleaned.csv")
df.head()

,product_name,product_url,test_type,product_description,remote_testing_available,Job_levels
0,Global Skills Development Report,https://www.shl.com/products/product-catalog/v...,"['Ability & Aptitude', 'Assessment Exercises',...",This report is designed to be given to individ...,Yes,"['Director', 'Entry-Level', 'Executive', 'Gene..."
1,ADO.NET (New),https://www.shl.com/products/product-catalog/v...,['Knowledge & Skills'],Multi-choice test that measures the knowledge ...,Yes,"['Mid-Professional', 'Professional Individual ..."
2,Accounts Receivable Simulation (New),https://www.shl.com/products/product-catalog/v...,['Simulations'],Simulated data entry test that measures the ab...,Yes,"['Entry-Level', 'Graduate', 'Mid-Professional'..."
3,Accounts Receivable (New),https://www.shl.com/products/product-catalog/v...,['Knowledge & Skills'],Multiple-choice test that measures the knowled...,Yes,"['Entry-Level', 'Graduate', 'Mid-Professional'..."
4,Accounts Payable Simulation (New),https://www.shl.com/products/product-catalog/v...,['Simulations'],Simulated data entry test that measures the ab...,Yes,"['Entry-Level', 'Graduate', 'Mid-Professional'..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   product_name              377 non-null    object
 1   product_url               377 non-null    object
 2   test_type                 377 non-null    object
 3   product_description       377 non-null    object
 4   remote_testing_available  377 non-null    object
 5   Job_levels                377 non-null    object
dtypes: object(6)
memory usage: 17.8+ KB


In [6]:
documents = []
metadata = []

for _, row in df.iterrows():
    text = f"""
    Product Name: {row['product_name']}
    Test Type: {row['test_type']}
    Job Levels: {row['Job_levels']}
    Remote Testing: {row['remote_testing_available']}
    Description: {row['product_description']}
    """
    documents.append(text)

    metadata.append({
        "product_name": row["product_name"],
        "product_url": row["product_url"],
        "test_type": row["test_type"],
        "product_description": row["product_description"],
        "remote_testing_available": row["remote_testing_available"],
        "Job_levels": row["Job_levels"]
    })


In [7]:
# Create embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(documents, show_progress_bar=True).astype("float32")

Batches: 100%|██████████| 12/12 [00:07<00:00,  1.67it/s]


In [8]:
# Build FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [9]:
faiss.write_index(index, "vector.index")
with open("vector_texts.pkl", "wb") as f:
    pickle.dump(metadata, f)

print("✅ FAISS index and metadata rebuilt correctly")

✅ FAISS index and metadata rebuilt correctly
